In [1]:
import requests, bs4
import mysql.connector
from lxml import html
from urllib.request import Request,urlopen
from urllib.parse import urlencode,quote_plus
from bs4 import BeautifulSoup
from xml.etree.ElementTree import parse
from datetime import datetime
from ServiceKey import public_serviceKey, mySQL_Connect

mydb = mySQL_Connect()
# 2021.10.07 데이터 추출 완료 - 31140 데이터가 대부분 없음, InterfaceError: Python type ResultSet cannot be converted > updowncd 값 2에 의한 에러
cursor = mydb.cursor(prepared=True)
citycode = '35320'

query1 = ("select citycode from public_busctycodelist where citycode >= {}".format(citycode))

cursor.execute(query1)

location_list1 = []

for a in cursor:
    location_list1.append(a)

now = datetime.now()
nowDate = now.strftime('%Y%m%d')

b_cnt = 0
for z in location_list1:
    if b_cnt == 1:
        break
        
    location = z
    query2 = ("select distinct routeid from public_busroute_info where citycode = '{}'".format(location[0]))
    cursor.execute(query2)

    location_list2 = []

    for b in cursor:
        location_list2.append(b)

    cnt = 1
    for i in location_list2:
        routeId = i

        characters = "(',')"

        join_data = ''.join( x for x in routeId if x not in characters)
        print("조회날짜 : {} | location : {} | cnt : {} | start {} public_busroute_info".format(nowDate,location[0],cnt,join_data))

        url = 'http://openapi.tago.go.kr/openapi/service/BusRouteInfoInqireService/getRouteAcctoThrghSttnList'
        queryParams = '?' + urlencode({
            quote_plus('serviceKey') : public_serviceKey(),
            quote_plus('cityCode') : location[0],
            quote_plus('pageNo') : '1',
            quote_plus('numOfRows') : '10000',
            quote_plus('routeId') : join_data
        })

        response = requests.get(url + queryParams).text.encode('utf-8')
        xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')

        rows = xmlobj.findAll('item')
        resultCode = xmlobj.findAll('resultCode')
        if resultCode[0].text == '99':
            print("location {} | {} 번째부터 조회수 제한이 걸렸습니다.".format(location[0],join_data))
            mydb.close()
            b_cnt += 1
            break
        if rows == []:
            print("location {} | {} 데이터가 없습니다.".format(location[0],join_data))
            cnt += 1
            continue
        cnt += 1
        for j in rows:
            item = j
            lat = item.findAll('gpslati')                
            lng = item.findAll('gpslong')
            nodeid = item.findAll('nodeid')
            nodenm = item.findAll('nodenm')
            nodeno = item.findAll('nodeno')
            if nodeno == []:
                nodeno = None
            else:
                nodeno = nodeno[0].text
            nodeord = item.findAll('nodeord')
            routeid = item.findAll('routeid')  
            updowncd = item.findAll('updowncd')
            
            if updowncd == []:
                updowncd = None
            elif updowncd[0].text == '0':
                updowncd = '상행'
            elif updowncd[0].text == '1':
                updowncd = '하행'
            elif updowncd[0].text == '2':
                updowncd = '2'

            sql = "INSERT INTO public_busroute_detail (lat, lng, nodeid, nodenm, nodeno, nodeord, routeid, updowncd, citycode) VALUES (?,?,?,?,?,?,?,?,?)"
            val = (lat[0].text, lng[0].text, nodeid[0].text, nodenm[0].text, nodeno, nodeord[0].text, routeid[0].text, updowncd, location[0])

            cursor.execute(sql, val)
            mydb.commit()

조회날짜 : 20211007 | location : 35040 | cnt : 1 | start TSB309000001 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 2 | start TSB309000002 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 3 | start TSB309000003 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 4 | start TSB309000004 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 5 | start TSB309000005 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 6 | start TSB309000006 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 7 | start TSB309000007 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 8 | start TSB309000008 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 9 | start TSB309000009 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 10 | start TSB309000010 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 11 | start TSB309000011 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt 

조회날짜 : 20211007 | location : 35040 | cnt : 95 | start TSB309000135 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 96 | start TSB309000136 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 97 | start TSB309000138 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 98 | start TSB309000139 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 99 | start TSB309000140 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 100 | start TSB309000141 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 101 | start TSB309000156 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 102 | start TSB309000161 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 103 | start TSB309000162 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 104 | start TSB309000163 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 105 | start TSB309000164 public_busroute_info
조회날짜 : 20211007 | location

조회날짜 : 20211007 | location : 35040 | cnt : 188 | start TSB309000289 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 189 | start TSB309000290 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 190 | start TSB309000291 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 191 | start TSB309000292 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 192 | start TSB309000293 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 193 | start TSB309000294 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 194 | start TSB309000295 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 195 | start TSB309000296 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 196 | start TSB309000297 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 197 | start TSB309000298 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 198 | start TSB309000299 public_busroute_info
조회날짜 : 20211007 | loc

조회날짜 : 20211007 | location : 35040 | cnt : 282 | start TSB309000451 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 283 | start TSB309000452 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 284 | start TSB309000453 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 285 | start TSB309000454 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 286 | start TSB309000456 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 287 | start TSB309000458 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 288 | start TSB309000460 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 289 | start TSB309000461 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 290 | start TSB309000463 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 291 | start TSB309000464 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 292 | start TSB309000468 public_busroute_info
조회날짜 : 20211007 | loc

조회날짜 : 20211007 | location : 35040 | cnt : 375 | start TSB309000211 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 376 | start TSB309000218 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 377 | start TSB309000219 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 378 | start TSB309000223 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 379 | start TSB309000224 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 380 | start TSB309000225 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 381 | start TSB309000234 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 382 | start TSB309000236 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 383 | start TSB309000238 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 384 | start TSB309000239 public_busroute_info
조회날짜 : 20211007 | location : 35040 | cnt : 385 | start TSB309000240 public_busroute_info
조회날짜 : 20211007 | loc

...

조회날짜 : 20211007 | location : 35050 | cnt : 91 | start NWB310001131 public_busroute_info
조회날짜 : 20211007 | location : 35050 | cnt : 92 | start NWB310001132 public_busroute_info
조회날짜 : 20211007 | location : 35050 | cnt : 93 | start NWB310001133 public_busroute_info
조회날짜 : 20211007 | location : 35050 | cnt : 94 | start NWB310001134 public_busroute_info
조회날짜 : 20211007 | location : 35050 | cnt : 95 | start NWB310001136 public_busroute_info
조회날짜 : 20211007 | location : 35050 | cnt : 96 | start NWB310001137 public_busroute_info
조회날짜 : 20211007 | location : 35050 | cnt : 97 | start NWB310001139 public_busroute_info
조회날짜 : 20211007 | location : 35050 | cnt : 98 | start NWB310001140 public_busroute_info
조회날짜 : 20211007 | location : 35050 | cnt : 99 | start NWB310001141 public_busroute_info
조회날짜 : 20211007 | location : 35050 | cnt : 100 | start NWB310001144 public_busroute_info
조회날짜 : 20211007 | location : 35050 | cnt : 101 | start NWB310001148 public_busroute_info
조회날짜 : 20211007 | location : 3

조회날짜 : 20211007 | location : 35050 | cnt : 184 | start NWB310001191 public_busroute_info
조회날짜 : 20211007 | location : 35050 | cnt : 185 | start NWB310001009 public_busroute_info
조회날짜 : 20211007 | location : 35050 | cnt : 186 | start NWB310001014 public_busroute_info
조회날짜 : 20211007 | location : 35050 | cnt : 187 | start NWB310001037 public_busroute_info
조회날짜 : 20211007 | location : 35050 | cnt : 188 | start NWB310001042 public_busroute_info
조회날짜 : 20211007 | location : 35050 | cnt : 189 | start NWB310001087 public_busroute_info
조회날짜 : 20211007 | location : 35050 | cnt : 190 | start NWB310001119 public_busroute_info
조회날짜 : 20211007 | location : 35050 | cnt : 191 | start NWB310001163 public_busroute_info
조회날짜 : 20211007 | location : 35060 | cnt : 1 | start TSB311000001 public_busroute_info
조회날짜 : 20211007 | location : 35060 | cnt : 2 | start TSB311000002 public_busroute_info
조회날짜 : 20211007 | location : 35060 | cnt : 3 | start TSB311000003 public_busroute_info
조회날짜 : 20211007 | location 

조회날짜 : 20211007 | location : 35060 | cnt : 88 | start TSB311000128 public_busroute_info
조회날짜 : 20211007 | location : 35060 | cnt : 89 | start TSB311000129 public_busroute_info
조회날짜 : 20211007 | location : 35060 | cnt : 90 | start TSB311000130 public_busroute_info
조회날짜 : 20211007 | location : 35060 | cnt : 91 | start TSB311000131 public_busroute_info
조회날짜 : 20211007 | location : 35060 | cnt : 92 | start TSB311000132 public_busroute_info
조회날짜 : 20211007 | location : 35060 | cnt : 93 | start TSB311000133 public_busroute_info
조회날짜 : 20211007 | location : 35060 | cnt : 94 | start TSB311000134 public_busroute_info
조회날짜 : 20211007 | location : 35060 | cnt : 95 | start TSB311000135 public_busroute_info
조회날짜 : 20211007 | location : 35060 | cnt : 96 | start TSB311000136 public_busroute_info
조회날짜 : 20211007 | location : 35060 | cnt : 97 | start TSB311000137 public_busroute_info
조회날짜 : 20211007 | location : 35060 | cnt : 98 | start TSB311000139 public_busroute_info
조회날짜 : 20211007 | location : 350

조회날짜 : 20211007 | location : 35060 | cnt : 181 | start TSB311000171 public_busroute_info
조회날짜 : 20211007 | location : 35060 | cnt : 182 | start TSB311000172 public_busroute_info
조회날짜 : 20211007 | location : 35060 | cnt : 183 | start TSB311000174 public_busroute_info
조회날짜 : 20211007 | location : 35060 | cnt : 184 | start TSB311000183 public_busroute_info
조회날짜 : 20211007 | location : 35060 | cnt : 185 | start TSB311000188 public_busroute_info
조회날짜 : 20211007 | location : 35060 | cnt : 186 | start TSB311000191 public_busroute_info
조회날짜 : 20211007 | location : 35060 | cnt : 187 | start TSB311000198 public_busroute_info
조회날짜 : 20211007 | location : 35060 | cnt : 188 | start TSB311000212 public_busroute_info
조회날짜 : 20211007 | location : 35060 | cnt : 189 | start TSB311000222 public_busroute_info
조회날짜 : 20211007 | location : 35060 | cnt : 190 | start TSB311000234 public_busroute_info
조회날짜 : 20211007 | location : 35060 | cnt : 191 | start TSB311000235 public_busroute_info
조회날짜 : 20211007 | loc

조회날짜 : 20211007 | location : 35320 | cnt : 38 | start TSB313000082 public_busroute_info
조회날짜 : 20211007 | location : 35320 | cnt : 39 | start TSB313000084 public_busroute_info
조회날짜 : 20211007 | location : 35320 | cnt : 40 | start TSB313000085 public_busroute_info
조회날짜 : 20211007 | location : 35320 | cnt : 41 | start TSB313000086 public_busroute_info
조회날짜 : 20211007 | location : 35320 | cnt : 42 | start TSB313000087 public_busroute_info
조회날짜 : 20211007 | location : 35320 | cnt : 43 | start TSB313000088 public_busroute_info
조회날짜 : 20211007 | location : 35320 | cnt : 44 | start TSB313000090 public_busroute_info
조회날짜 : 20211007 | location : 35320 | cnt : 45 | start TSB313000091 public_busroute_info
조회날짜 : 20211007 | location : 35320 | cnt : 46 | start TSB313000092 public_busroute_info
조회날짜 : 20211007 | location : 35320 | cnt : 47 | start TSB313000094 public_busroute_info
조회날짜 : 20211007 | location : 35320 | cnt : 48 | start TSB313000098 public_busroute_info
조회날짜 : 20211007 | location : 353